# Pre-processing

This notebook contains pre-processing steps to create cleaned data in the `/data` folder from the raw datasets.

In [52]:
import pandas as pd
import os
from thefuzz import process
import subprocess

from src.utils.preprocess.location_extracter import extract_location
from src.utils.preprocess.preprocess import (
    preprocess_ba_beers,
    preprocess_ba_breweries,
    preprocess_ba_users,
    preprocess_ba_ratings,
    compute_gini_impurity,
    preprocess_rb_beers,
    preprocess_rb_beers_style,
    preprocess_rb_breweries,
    preprocess_rb_users,
    preprocess_rb_ratings,
)
from src.utils.preprocess.past_ratings import (
    get_past_ratings_count_and_average,
    get_past_ratings_counts_all_style,
)

In [2]:
os.makedirs("src/data", exist_ok=True)
os.makedirs("src/data/beer_advocate", exist_ok=True)
os.makedirs("src/data/matched_beer_data", exist_ok=True)
os.makedirs("src/data/rate_beer", exist_ok=True)

# Downloading data

The dataset is stored on a remote Google Drive folder. The following cell with execute a python script that will download the files and store then in a folder `raw_data` at the root of the repository.

In [3]:
%run src/scripts/download.py {os.path.join(os.getcwd(), 'raw_data')}

Downloading...
From (original): https://drive.google.com/uc?id=1IqcAJtYrDB1j40rBY5M-PGp6KNX-E3xq
From (redirected): https://drive.google.com/uc?id=1IqcAJtYrDB1j40rBY5M-PGp6KNX-E3xq&confirm=t&uuid=13fe231f-8ac7-46f7-a1d0-348aec24198e
To: /Users/martinlebras/Documents/Cours/EPFL/Projets/ada-2024-project-pdfromscratch/raw_data/beer_advocate/BeerAdvocate.tar.gz
100%|██████████| 1.67G/1.67G [01:30<00:00, 18.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1SdScOOuA219GeA6jP6CTkj98FP_X0DMe
From (redirected): https://drive.google.com/uc?id=1SdScOOuA219GeA6jP6CTkj98FP_X0DMe&confirm=t&uuid=ce13ce71-6121-48ec-94fe-aa09e9c7a9b4
To: /Users/martinlebras/Documents/Cours/EPFL/Projets/ada-2024-project-pdfromscratch/raw_data/matched_beer_data/matched_beer_data.tar.gz
100%|██████████| 454M/454M [00:20<00:00, 21.6MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1vt-CTz6Ni8fPTIkHehW9Mm0RPMpvkH3a
From (redirected): https://drive.google.com/uc?id=1vt-CTz6Ni8fPTIkHeh

# Beer Advocate

In [4]:
df_ba_beers_styles = pd.read_csv("src/data/beers_styles.csv")

## Beers

In [6]:
df_ba_beers = pd.read_csv("raw_data/beer_advocate/beers.csv")
df_ba_beers = preprocess_ba_beers(df_ba_beers, df_ba_beers_styles)
df_ba_beers.to_csv("src/data/beer_advocate/beers.csv", index=False)

## Breweries

In [8]:
df_ba_breweries = pd.read_csv("raw_data/beer_advocate/breweries.csv")
df_ba_breweries = preprocess_ba_breweries(df_ba_breweries)
df_ba_breweries.to_csv("src/data/beer_advocate/breweries.csv", index=False)

## Users

In [10]:
df_ba_users = pd.read_csv("raw_data/beer_advocate/users.csv")
df_ba_users = preprocess_ba_users(df_ba_users)
df_ba_users.to_csv("src/data/beer_advocate/users.csv", index=False)

## Ratings & Reviews

On BeerAdvocate, users can either submit a *rating* or a *review*. The dataset contains two separate text files `ratings.txt` and `reviews.txt`. Ratings and reviews are formatted as (key, value) pairs on each line of a plain text file with empty lines to split different ratings/reviews. 

The program `txt_to_csv.cpp` is a C++ program that transform a plain text file in a `.csv` file, that can be more easily analyzed using Python and Pandas. The following cell will compile the program using a C++ compiler and run it on the `ratings.txt` file only. Indeed, after analyzing the overlapping between the two files, we found that `reviews.txt` is a subset of `ratings.txt`. Thus, we will discard `reviews.txt` and only keep `ratings.txt`.

In [11]:
subprocess.run(
    "g++ -std=c++17 -o src/scripts/txt_to_csv src/scripts/txt_to_csv.cpp", shell=True
)

subprocess.run(
    [
        "src/scripts/txt_to_csv",
        "raw_data/beer_advocate/ratings.txt",
        "raw_data/beer_advocate/ratings.csv",
    ]
)

CompletedProcess(args=['src/scripts/txt_to_csv', 'raw_data/beer_advocate/ratings.txt', 'raw_data/beer_advocate/ratings.csv'], returncode=0)

In [13]:
df_ba_ratings = pd.read_csv("raw_data/beer_advocate/ratings.csv")
df_ba_ratings = preprocess_ba_ratings(df_ba_ratings)

For several analyses, we will be looking for the average scores given by the user on all the beers and on different styles of beer at the time of rating. We will also require the current rating of the beer evaluated at the time of rating. The function `get_past_ratings_count_and_average` computes the count and average of ratings according to provided parameters :

|Parameter|Description|
|---|---|
|`sorting_columns` | Columns to sort to ensure the cumulative count and mean are done in the right order |
|`grouping_columns` | Columns forming the group within which we want to compute the count and mean |
|`name` | Prefix of the resulting columns |
|`shift` | If `True`, the count and mean will **not** include the current rating in the computation |

### Past ratings of each user

The next cell computes the number and the average of past ratings of the user that posted a given rating at the time of this rating.
$$
\begin{align*}
& \forall i \in \{0, 1, \dots, N\} \\ 
\mathcal S_i &= \{\verb|ratings[n]| | (\verb|ratings[i][user] == ratings[n][user]|) \land (\verb|ratings[i][date] > ratings[n][date]|) \} \\
\verb|count[i]| &= |\mathcal S_i| \\
\verb|average[i]| &= \frac{1}{|\mathcal S_i|} \sum_{\verb|rating| \in \mathcal S_i} \verb|rating[rating]|
\end{align*}
$$



In [15]:
df_ba_ratings = get_past_ratings_count_and_average(
    df_ba_ratings, ["user_id", "date"], ["user_id"], "user", shift=True
)

### Past ratings of each beer

The next cell computes the number and the average of past ratings of the beer that is rated in a given rating at the time of this rating.
$$
\begin{align*}
& \forall i \in \{0, 1, \dots, N\} \\ 
\mathcal S_i &= \{\verb|ratings[n]| | (\verb|ratings[i][beer] == ratings[n][beer]]|) \land (\verb|ratings[i][date] > ratings[n][date]|) \} \\
\verb|count[i]| &= |\mathcal S_i| \\
\verb|average[i]| &= \frac{1}{|\mathcal S_i|} \sum_{\verb|rating| \in \mathcal S_i} \verb|rating[rating]|
\end{align*}
$$



In [16]:
df_ba_ratings = get_past_ratings_count_and_average(
    df_ba_ratings, ["beer_id", "date"], ["beer_id"], "beer", shift=True
)

### Past ratings of each user for a specific beer style

The next cell computes the number and the average of past ratings of the user that posted a given rating at the time of this rating and only for beers of the same style as the one of the current rating.
$$
\begin{align*}
& \forall i \in \{0, 1, \dots, N\} \\ 
\mathcal S_i &= \{\verb|ratings[n]| |
    \\ & \quad\quad (\verb|ratings[i][user] == ratings[n][user]|)
    \\ & \quad\quad \land  (\verb|ratings[i][date] > ratings[n][date]|)
    \\ & \quad\quad \land (\verb|ratings[i][beer_style] == ratings[n][beer_style]|) 
\\ & \} \\
\verb|count[i]| &= |\mathcal S_i| \\
\verb|average[i]| &= \frac{1}{|\mathcal S_i|} \sum_{\verb|rating| \in \mathcal S_i} \verb|rating[rating]|
\end{align*}
$$

In [17]:
# Add global beer style to ratings
df_ba_ratings = df_ba_ratings.merge(
    df_ba_beers[["beer_id", "beer_global_style"]], on="beer_id", how="left"
)
df_ba_ratings = get_past_ratings_count_and_average(
    df_ba_ratings,
    ["user_id", "beer_global_style", "date"],
    ["user_id", "beer_global_style"],
    "user_beer_style",
    shift=False,
)

### Past ratings of each user for all beer styles

The next cell computes the number and the average of past ratings of the user that posted a given rating at the time of this rating, for all styles of beer.

Let $\mathcal B$ be the set of beer styles.

$$
\begin{align*}
& \forall i \in \{0, 1, \dots, N\}, \forall j \in \mathcal B\\ 
\mathcal S_{i, j} &= \{\verb|ratings[n]| |
    \\ & \quad\quad (\verb|ratings[i][user] == ratings[n][user]|)
    \\ & \quad\quad \land  (\verb|ratings[i][date] > ratings[n][date]|)
    \\ & \quad\quad \land (\verb|ratings[n][beer_style] == j|) 
\\ & \} \\
\verb|count[i][j]| &= |\mathcal S_{i, j}| \\
\verb|average[i][j]| &= \frac{1}{|\mathcal S_{i, j}|} \sum_{\verb|rating| \in \mathcal S_{i, j}} \verb|rating[rating]|
\end{align*}
$$

Thus this function will create $2 \times |\mathcal B|$ new columns with the counts and averages for each style of beer.

In [19]:
df_user_beer_style_past_ratings = get_past_ratings_counts_all_style(
    df_ba_ratings, "beer_global_style", "user_beer_style"
)
df_user_beer_style_past_ratings.to_csv(
    "src/data/beer_advocate/user_beer_style_past_ratings.csv", index=False
)

/var/folders/ng/n8p0hjyx5vg98_zrmqdwzxzw0000gn/T/ipykernel_38842/126769524.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_user_beer_style_past_ratings[modified_columns] = df_user_beer_style_past_ratings[modified_columns].fillna(0.0)


### User rating habits of different beer styles

The next cell computes the Gini impurity of each user with respect to the number of beers of each style they rated. The Gini impurity is in $[0, 1]$ where $0$ means that the user has rated only one style of beer and $1$ means that the user has rated as many beers of each style.

Let $\mathcal S$ be the set of beer styles, then the **normalized** Gini impurity $G$ is computed as

$$G=\frac{1 - \sum_{s \in \mathcal S} p_s^2}{1 - \frac{1}{|\mathcal S|}}$$

In [21]:
df_ba_ratings = compute_gini_impurity(df_ba_ratings, df_user_beer_style_past_ratings)

In [22]:
df_ba_ratings.sort_index(inplace=True)
df_ba_ratings.to_csv("src/data/beer_advocate/ratings.csv", index=False)

## Memory cleaning

In [23]:
del df_ba_beers
del df_ba_breweries
del df_ba_users
del df_ba_ratings

# RateBeer

## Beers

In [58]:
df_rb_beers = pd.read_csv("raw_data/rate_beer/beers.csv")
df_rb_beers = preprocess_rb_beers(df_rb_beers)
df_rb_beers = preprocess_rb_beers_style(df_rb_beers, df_ba_beers_styles)

Regarding the low number of beers that we were not able to match correctly to a global style, we decided to drop those beers.

In [59]:
print(
    f"[RateBeer] {df_rb_beers['beer_global_style'].isna().mean() * 100:.2f}% of beers have no global style"
)

[RateBeer] 1.67% of beers have no global style


In [60]:
df_rb_beers = df_rb_beers[~df_rb_beers["beer_global_style"].isna()]
df_rb_beers.to_csv("src/data/rate_beer/beers.csv", index=False)

## Breweries

In [62]:
df_rb_breweries = pd.read_csv("raw_data/rate_beer/breweries.csv")
df_rb_breweries = preprocess_rb_breweries(df_rb_breweries)
df_rb_breweries.to_csv("src/data/rate_beer/breweries.csv", index=False)

## Users

In [63]:
df_rb_users = pd.read_csv("raw_data/rate_beer/users.csv")

In [65]:
df_rb_users = pd.read_csv("raw_data/rate_beer/users.csv")
df_rb_users = preprocess_rb_users(df_rb_users)
df_rb_users.to_csv("src/data/rate_beer/users.csv", index=False)

## Ratings & Reviews

Since there are only ratings on Rate Beer, the files `ratings.txt` and `reviews.txt` are strictly identical. We discard `reviews.txt` and parse `ratings.txt` using the `txt_to_csv.cpp` utility.

In [ ]:
subprocess.run(
    [
        "src/scripts/txt_to_csv",
        "raw_data/rate_beer/ratings.txt",
        "raw_data/rate_beer/ratings.csv",
    ]
)

In [66]:
df_rb_ratings = pd.read_csv("raw_data/rate_beer/ratings.csv")
df_rb_ratings = preprocess_rb_ratings(df_rb_ratings, df_rb_beers)

Since not all criterias are rated on the same scale, we rescaled everything on the same 5-star scale.

In [70]:
df_rb_ratings[["overall", "aroma", "appearance", "palate", "taste"]] = (
    5
    * df_rb_ratings[["overall", "aroma", "appearance", "palate", "taste"]]
    / df_rb_ratings[["overall", "aroma", "appearance", "palate", "taste"]].max()
)

### Past ratings of each user

In [39]:
df_rb_ratings = get_past_ratings_count_and_average(
    df_rb_ratings, ["user_id", "date"], ["user_id"], "user", shift=True
)

### Past ratings of each beer

In [40]:
df_rb_ratings = get_past_ratings_count_and_average(
    df_rb_ratings, ["beer_id", "date"], ["beer_id"], "beer", shift=True
)

### Past ratings of each user for all beer styles

In [41]:
df_rb_ratings = df_rb_ratings.merge(
    df_rb_beers[["beer_id", "beer_global_style"]], on="beer_id", how="left"
)
df_rb_ratings = get_past_ratings_count_and_average(
    df_rb_ratings,
    ["user_id", "beer_global_style", "date"],
    ["user_id", "beer_global_style"],
    "user_beer_style",
    shift=False,
)

In [42]:
df_user_beer_style_past_ratings = get_past_ratings_counts_all_style(
    df_rb_ratings, "beer_global_style", "user_beer_style"
)
df_user_beer_style_past_ratings.to_csv(
    "src/data/rate_beer/user_beer_style_past_ratings.csv", index=False
)

/var/folders/ng/n8p0hjyx5vg98_zrmqdwzxzw0000gn/T/ipykernel_38842/126769524.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_user_beer_style_past_ratings[modified_columns] = df_user_beer_style_past_ratings[modified_columns].fillna(0.0)


### User rating habits of different beer styles

In [43]:
df_rb_ratings = compute_gini_impurity(df_rb_ratings, df_user_beer_style_past_ratings)

In [44]:
df_rb_ratings.sort_index(inplace=True)
df_rb_ratings.to_csv("src/data/rate_beer/ratings.csv", index=False)

## Memory cleaning

In [45]:
del df_rb_beers
del df_rb_breweries
del df_rb_users
del df_rb_ratings

# Matched Beer Data

## Beers

In [46]:
df_ma_beers = pd.read_csv("raw_data/matched_beer_data/beers.csv", header=[0, 1])
df_ma_beers = df_ma_beers[
    [("ba", "beer_id"), ("rb", "beer_id"), ("scores", "diff"), ("scores", "sim")]
]
df_ma_beers.to_csv("src/data/matched_beer_data/beers.csv", index=False)

## Breweries

In [47]:
df_ma_breweries = pd.read_csv("raw_data/matched_beer_data/breweries.csv", header=[0, 1])
df_ma_breweries = df_ma_breweries.rename(
    columns={
        "id": "brewery_id",
    },
    level=1,
)
df_ma_breweries = df_ma_breweries[
    [("ba", "brewery_id"), ("rb", "brewery_id"), ("scores", "diff"), ("scores", "sim")]
]
df_ma_breweries.to_csv("src/data/matched_beer_data/breweries.csv", index=False)

## Users

In [48]:
df_ma_users = pd.read_csv("raw_data/matched_beer_data/users.csv", header=[0, 1])
df_ma_users = df_ma_users[[("ba", "user_id"), ("rb", "user_id")]]
df_ma_users.to_csv("src/data/matched_beer_data/users.csv", index=False)

In [49]:
df_ma_users_approx = pd.read_csv(
    "raw_data/matched_beer_data/users_approx.csv", header=[0, 1]
)
df_ma_users_approx = df_ma_users_approx[
    [("ba", "user_id"), ("rb", "user_id"), ("scores", "sim")]
]
df_ma_users_approx.to_csv("src/data/matched_beer_data/users_approx.csv", index=False)

## Ratings & Reviews

In [50]:
df_ma_ratings = pd.read_csv("raw_data/matched_beer_data/ratings.csv", header=[0, 1])
df_ma_ratings = df_ma_ratings[
    [
        ("ba", "beer_id"),
        ("ba", "user_id"),
        ("rb", "beer_id"),
        ("rb", "user_id"),
    ]
]
df_ma_ratings.to_csv("src/data/matched_beer_data/ratings.csv", index=False)

## Memory cleaning

In [51]:
del df_ma_beers
del df_ma_breweries
del df_ma_users
del df_ma_users_approx
del df_ma_ratings